# MADS-Deep Learning
---
## Portfolio Examination Part 1
#### Janosch Höfer, 938969

## Table of contents

- [Imports](#imports) <br>
- [1. Exercise](#task1) <br>
- [2. Exercise](#task2) <br>
- [3. Exercise](#task3) <br>
- [4. Exercise](#task4) <br>
-[References](#ref)<br>

<a id='imports'></a>
## Imports

In [ ]:
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split

torch.manual_seed(42)

<a id='task1'></a>
## Exercise 1
Given a perceptron with weights $(0.1, 0.4, 0.6, 0.7)$ and bias $0.2$, compute the output for the tensor $\begin{pmatrix}1 & 0 & 1 & 0\\0.1 & 0.2 & 0.1 & 0.2\end{pmatrix}$ of dimensions (dataset, features).

In [ ]:
weights = torch.tensor([0.1, 0.4, 0.6, 0.7])
bias = torch.tensor(0.2)
vector = torch.tensor([[1, 0, 1, 0], [0.1, 0.2, 0.1, 0.2]])

In [ ]:
def perceptron_predict(
    input_t: torch.Tensor, weights: torch.Tensor, bias: torch.Tensor
) -> torch.Tensor:
    return torch.matmul(input_t, weights) + bias

In [ ]:
output_1 = perceptron_predict(vector, weights, bias)
output_1

The result of the perceptron is a tensor with dimensionality (dataset). The results are 0.9 and 0.49.

<a id='task2'></a>
## Exercise 2
The vacation platform JourneyAdvisor wants to apply deep learning in their recommender engine,
that recommends points of interest to users based on their user account properties and previously
visited places. The catalog of the platform contains $14,467$ points of interest. Users can check-in at
such places using their phones. The platform has $1,989,345$ users. When users register, they enter
their birthday, a payment method and their home address.<br>
1. Propose a list of features, suitable for the recommendation task. Explain your choice!

Suitable features could be:
* Age
* Number of visits
* Payment method (as embeddings)
* Home address (postal code)

Instead of using the birthday, we can calculate the age of the user. This has the advantage, that age is not only a continuous value. It also is a smaller number than the birthday. The number of visits is also a continuous value that should be used for recommendations. If a user visits types of places more often, we can recommend places that users with similar preferences have visited. The payment method could be an interesting feature. Someone who uses Apple Pay would probably prefer to visit places, that offer this payment method. Whether a place offers Apple Pay can be inferred from other users who have visited that place. The difficulty here is that the payment method is a categorical value. To make this feature usable by our neural network, it has to be transformed, either using One-Hot Encoding or Embeddings. The last feature is the home address, more precisely the zip code. Here we have the same problem, that the zip code is a categorical value. But fortunately we have multiple solutions for that problem. The first approach uses continuous data that can be assigned to the various zip codes, e.g. average salary, crime-rate, house prices, etc. The second approach uses the latitude and longitude. In both approaches we can further fine tune the data, by changing the granularity of the zip code to look at districts, cities or states.

2. Describe a tensor to model the data for JourneyAdvisor. Describe its dimensionality.

$T\begin{matrix}( POI,&&    User,    &&& features)\end{matrix}$<br>
$T\begin{matrix}( 14,467,& 1,989,345, & 4&)\end{matrix}$

3. How many entries does the tensor have?

In [ ]:
print(f"{14467 * 1989345 * 4:,d}")

With the above dimensionality the total entries are 115,119,416,460. That's 115 Billion entries. Each additional feature increases it by ca. 28 Billion.

<a id='task3'></a>
## Exercise 3
Familiarize yourself with the SMOTE [[1]](#1) algorithm. In your own words, describe the use-case of the
SMOTE. Among others, address these points:
1. In which situations can it be useful (explain in general and provide three examples)?
2. What is its fundamental idea?
3. How is SMOTE different from oversampling with replacement?
Exercise 4.

<a id='task4'></a>
## Exercise 4
Create a Jupyter Notebook to solve the following machine learning task in Python, using PyTorch
(and other suitable libraries):
1. Load and arrange the dataset *portfolio_data_sose_2022.csv*. It has two features, feature_1 and feature_2, and a target variable target.

In [ ]:
raw_df = pd.read_csv("data/portfolio_data_sose_2022.csv")
print(
    f"Number of rows: \t{raw_df.shape[0]} \n"
    f"Number of columns: \t{raw_df.shape[1]}\n"
    f"Number of targets: \t{raw_df['target'].nunique()}"
)

In [ ]:
raw_df.head()

In [ ]:
data = raw_df.iloc[:, :2]
labels = raw_df["target"]
data_train, data_test, y_train, y_test = train_test_split(
    data, labels, test_size=0.33, random_state=42, shuffle=True, stratify=labels
)

In [ ]:
y_train.value_counts(), y_test.value_counts()

2. Describe the class distribution.

The classes are highly imbalanced. Out of the 10.000 entries target 0 accounts for 9.900. 

In [ ]:
raw_df["target"].value_counts()

In [ ]:
sns.countplot(data=raw_df, x="target")

3. Plot the data.

In [ ]:
sns.jointplot(data=raw_df, x="feature_1", y="feature_2", hue="target", kind="scatter")

In [ ]:
sns.relplot(x="feature_1", y="feature_2", data=raw_df, col="target")

4. Create a simple (single layer) neural network with two output neurons, one for each of the two classes 0 and 1 (i.e. use a multiclass classification setup).

In [ ]:
class NNClassifier(nn.Module):
    def __init__(self, input_size: int):
        super().__init__()
        self.lin1 = nn.Linear(input_size, 1)

    def forward(self, x):
        pass

    def predict(self, x: torch.Tensor):
        pass

<a id='ref'></a>
## References
<a id='1'>N. V. Chawla, K. W. Bowyer, L. O. Hall, and W. P. Kegelmeyer, “Smote: Synthetic minority
over-sampling technique,” Journal of Artificial Intelligence Research, vol. 16, pp. 321–357, 2002.</a>